A general overview and code of how the merging of the calculated points goes.

For the two groupings we're merging, the first, varaibles with no number at the end, we have 3 parings of ids where we have the information of their combined cost and combined points. For the second group, variables that end in `2`, there are four groups of ids, costs, and points. We're working in deminsions here, so it's important to keep in mind that as long as the indicies of the values associated with the people are legal, then we're good.

In [4]:
import numpy as np
import itertools

In [13]:
ids = np.array([np.arange(3), np.arange(3) + 3]).T #ids.shape == (3,2)
ids

array([[0, 3],
       [1, 4],
       [2, 5]])

In [14]:
ids2 = np.array([np.arange(4) + 6, np.arange(4) + 10]).T #ids2.shape == (4,2)
ids2

array([[ 6, 10],
       [ 7, 11],
       [ 8, 12],
       [ 9, 13]])

In [62]:
ids_comb = np.array([np.concatenate((x,y)) for x,y in list(itertools.product(ids, ids2))])
ids_comb

array([[ 0,  3,  6, 10],
       [ 0,  3,  7, 11],
       [ 0,  3,  8, 12],
       [ 0,  3,  9, 13],
       [ 1,  4,  6, 10],
       [ 1,  4,  7, 11],
       [ 1,  4,  8, 12],
       [ 1,  4,  9, 13],
       [ 2,  5,  6, 10],
       [ 2,  5,  7, 11],
       [ 2,  5,  8, 12],
       [ 2,  5,  9, 13]])

`costs` and `costs2` variables are created with random numbers insequential order

In [5]:
costs = np.array([5,7,9])
costs

array([5, 7, 9])

In [32]:
costs2 = np.array([2,3,5,8])
costs2

array([2, 3, 5, 8])

We can see that the array below has all the different ways we can combined those cost numbers

In [7]:
cost_comb = np.array(list(itertools.product(costs, costs2)))
cost_comb

array([[5, 2],
       [5, 3],
       [5, 5],
       [5, 8],
       [7, 2],
       [7, 3],
       [7, 5],
       [7, 8],
       [9, 2],
       [9, 3],
       [9, 5],
       [9, 8]])

In [ ]:
And now we can see that those values have been added together. Now we know the complete cost of those players

In [8]:
cost_sum = cost_comb.sum(axis=1)
cost_sum

array([ 7,  8, 10, 13,  9, 10, 12, 15, 11, 12, 14, 17])

In [9]:
points = np.array([3,4,6])
points

array([3, 4, 6])

In [10]:
points2 = np.array([2,4,6,7])
points2

array([2, 4, 6, 7])

Like above, were calculating the product of the `points` and `points2` variables. Key point is that `itertools.product` returns the values in the same order. If that wasn't the case, this fails.

In [11]:
points_comb = np.array(list(itertools.product(points, points2)))
points_comb

array([[3, 2],
       [3, 4],
       [3, 6],
       [3, 7],
       [4, 2],
       [4, 4],
       [4, 6],
       [4, 7],
       [6, 2],
       [6, 4],
       [6, 6],
       [6, 7]])

In [12]:
points_sum = points_comb.sum(axis=1)
points_sum

array([ 5,  7,  9, 10,  6,  8, 10, 11,  8, 10, 12, 13])

For the third time, we're grouping the ids together in the same order. The concatenate makes the array have only two dimensions which is what we want for now.

In [18]:
test_salaries = np.array([9,10,11,12])
test_salaries

array([ 9, 10, 11, 12])

When we run the `broadcast_to`, we can see that with the following shape, each row is a test for each salary. Keep that in mind.

In [21]:
test_salaries_full = np.broadcast_to(test_salaries, (cost_sum.size, test_salaries.size)).T #(cost_sum.size, test_salaries.size) ensures the correct dimensions
test_salaries_full

array([[ 9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9],
       [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
       [11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11],
       [12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12]])

In [52]:
ids_comb_full = np.broadcast_to(ids_comb, (test_salaries.size, points_sum.size, 4))
ids_comb_full

array([[[ 0,  3,  6, 10],
        [ 0,  3,  7, 11],
        [ 0,  3,  8, 12],
        [ 0,  3,  9, 13],
        [ 1,  4,  6, 10],
        [ 1,  4,  7, 11],
        [ 1,  4,  8, 12],
        [ 1,  4,  9, 13],
        [ 2,  5,  6, 10],
        [ 2,  5,  7, 11],
        [ 2,  5,  8, 12],
        [ 2,  5,  9, 13]],

       [[ 0,  3,  6, 10],
        [ 0,  3,  7, 11],
        [ 0,  3,  8, 12],
        [ 0,  3,  9, 13],
        [ 1,  4,  6, 10],
        [ 1,  4,  7, 11],
        [ 1,  4,  8, 12],
        [ 1,  4,  9, 13],
        [ 2,  5,  6, 10],
        [ 2,  5,  7, 11],
        [ 2,  5,  8, 12],
        [ 2,  5,  9, 13]],

       [[ 0,  3,  6, 10],
        [ 0,  3,  7, 11],
        [ 0,  3,  8, 12],
        [ 0,  3,  9, 13],
        [ 1,  4,  6, 10],
        [ 1,  4,  7, 11],
        [ 1,  4,  8, 12],
        [ 1,  4,  9, 13],
        [ 2,  5,  6, 10],
        [ 2,  5,  7, 11],
        [ 2,  5,  8, 12],
        [ 2,  5,  9, 13]],

       [[ 0,  3,  6, 10],
        [ 0,  3,  7, 11],
      

In [53]:
cost_sum_full = np.broadcast_to(cost_sum,(test_salaries.size, cost_sum.size))
cost_sum_full

array([[ 7,  8, 10, 13,  9, 10, 12, 15, 11, 12, 14, 17],
       [ 7,  8, 10, 13,  9, 10, 12, 15, 11, 12, 14, 17],
       [ 7,  8, 10, 13,  9, 10, 12, 15, 11, 12, 14, 17],
       [ 7,  8, 10, 13,  9, 10, 12, 15, 11, 12, 14, 17]])

In [54]:
points_sum_full = np.broadcast_to(points_sum,(test_salaries.size, points_sum.size))
points_sum_full

array([[ 5,  7,  9, 10,  6,  8, 10, 11,  8, 10, 12, 13],
       [ 5,  7,  9, 10,  6,  8, 10, 11,  8, 10, 12, 13],
       [ 5,  7,  9, 10,  6,  8, 10, 11,  8, 10, 12, 13],
       [ 5,  7,  9, 10,  6,  8, 10, 11,  8, 10, 12, 13]])

At this point, note the shapes of cost_sum_full, points_sum_full, test_salaries_full, and ids_comb_full:

In [66]:
print("cost_sum_full:     ", cost_sum_full.shape)
print("points_sum_full:   ", points_sum_full.shape)
print("test_salaries_full:", test_salaries_full.shape)
print("ids_comb_full:     ", ids_comb_full.shape)

cost_sum_full:      (4, 12)
points_sum_full:    (4, 12)
test_salaries_full: (4, 12)
ids_comb_full:      (4, 12, 4)


The key work before was to get these deminsions to match up so we can do direct comparison. As an example, we can pick (x,y) coords as (2,4) and see that the information matches up. Remember, the reason that `ids_comb_full` has that 3rd dimension is because we're combining groups of 4 player ids.

In [60]:
x,y = 2,4
print("Indexes: ", ids_comb_full[x][y])
print("Cost: ", cost_sum_full[x][y])
print("Points: ", points_sum_full[x][y])

Indexes:  [ 1  4  6 10]
Cost:  9
Points:  6


In [25]:
valids = cost_sum_full <= test_salaries_full
valids

array([[ True,  True, False, False,  True, False, False, False, False,
        False, False, False],
       [ True,  True,  True, False,  True,  True, False, False, False,
        False, False, False],
       [ True,  True,  True, False,  True,  True, False, False,  True,
        False, False, False],
       [ True,  True,  True, False,  True,  True,  True, False,  True,
         True, False, False]])

In [26]:
possibilities = points_sum_full * valids
possibilities

array([[ 5,  7,  0,  0,  6,  0,  0,  0,  0,  0,  0,  0],
       [ 5,  7,  9,  0,  6,  8,  0,  0,  0,  0,  0,  0],
       [ 5,  7,  9,  0,  6,  8,  0,  0,  8,  0,  0,  0],
       [ 5,  7,  9,  0,  6,  8, 10,  0,  8, 10,  0,  0]])

In [27]:
top_inds = possibilities.argmax(axis=1)
top_inds

array([1, 2, 2, 6])

In [28]:
row_selectors = np.arange(test_salaries.size)
row_selectors

array([0, 1, 2, 3])

In [29]:
max_points = points_sum_full[row_selectors, top_inds]
max_points

array([ 7,  9,  9, 10])

In [30]:
max_costs = cost_sum_full[row_selectors, top_inds]
max_costs

array([ 8, 10, 10, 12])

In [31]:
max_inds = ids_comb_full[row_selectors, top_inds]
max_inds

array([[ 0,  3,  7, 11],
       [ 0,  3,  8, 12],
       [ 0,  3,  8, 12],
       [ 1,  4,  8, 12]])

In [64]:
test_salaries

array([ 9, 10, 11, 12])

Welcome to the bottom! What we have now are three arrays: max_points, max_costs, and max_inds.
I showed test_salaries again to remined what's going on.

`test_salaries[0]` is 9, meaning that for the 0th position in the max_points, max_costs, and max_inds correlates to the best group of players whose combined salary is less than 9.

Players with the indicies `[0,3,7,11]` have a combined cost of 8, and points of 7. That grouping beats everyone else.

Again, this is a tiny example, with very small amount of data. Hopefully this give a little more sense of what we're doing with the merge.